In [1]:
import spacy
import hashlib
import pickle

In [2]:
nlp = spacy.load('en_coref_lg')

In [3]:
id2Word = dict()
def getID(s):
    uid = hashlib.md5(s.encode()).hexdigest()[-6:]
    id2Word[uid] = s
    return uid

def clean_text(text):
    return text.replace("\n", " ")

def resolve_coref(text):
    """
    :param text String of text where to resolve coreference.
    :return: A text String where the coreference has been resolved.
    """
    doc = nlp(text)
    return doc._.coref_resolved    

def get_entities(sent):
    """
    :param sent: A parsed sentence by Spacy
    :return: all the entities present in the sentence
    """
    return [ent for ent in sent.ents if ent.label_ in ['PERSON', 'ORG', 'DATE', 'NORP', 'EVENT']]

In [4]:
with open('test.txt', 'r') as myfile:
    text=myfile.read()#.replace('\n', '')

In [5]:
text = clean_text(text)
text = resolve_coref(text)

In [6]:
lines = []
for sentence in nlp(text).sents:
    sentence = sentence.as_doc()
    sent = " ".join([str(t) for t in sentence])
    ents = get_entities(sentence)
    for e1 in ents:
        for e2 in ents:
            if e1 != e2:
                json_sent = {
                    'sentence': sent,
                    'head': {'word': e1.text, 'id': getID(e1.text)},
                    'tail': {'word': e2.text, 'id': getID(e2.text)},
                    'relation': 'NA' #set to NA as it has to be disovered
                }
                lines.append(json_sent)

In [7]:
def write(data,outfile="test.json"):
    import json
    with open('test.json', 'w') as outfile:
        json.dump(data, outfile)
write(lines)

In [8]:
pickle.dump(id2Word, open('id2Word.pkl', 'wb'))

In [10]:
#pickle.load(open('id2Word.pkl', 'rb'))